In [2]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from statsmodels.api import OLS
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP.models import \
(Stepwise,
sklearn_selected,
sklearn_selection_path)
!pip install l0bnb
from l0bnb import fit_path

In [4]:
Hitters = load_data('Hitters')

In [5]:
Hitters

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [6]:
np.isnan(Hitters['Salary']).sum()

59

In [7]:
#dropna() method of data frames removes all of the rows that have missing values in any variable
Hitters = Hitters.dropna();
Hitters.shape

(263, 20)

In [8]:
def nCp(sigma2, estimator, X, Y):
    "Negative Cp statistic" 
    #sklearn tries to maximize a score 
    n, p = X.shape
    Yhat = estimator.predict(X)
    RSS = np.sum((Y - Yhat)**2)
    return -(RSS + 2*p*sigma2) / n 
    #sigma^2 is variance estimated using the full model containing all predictors

In [9]:
design = MS(Hitters.columns.drop('Salary')).fit(Hitters)
design

C:\Users\Asus\anaconda3\Lib\site-packages\ISLP\models\columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
C:\Users\Asus\anaconda3\Lib\site-packages\ISLP\models\columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
C:\Users\Asus\anaconda3\Lib\site-packages\ISLP\models\columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
C:\Users\Asus\anaconda

ModelSpec(terms=Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'NewLeague'],
      dtype='object'))

In [22]:
Y = np.array(Hitters['Salary'])
X = design.transform(Hitters)
X

,intercept,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League[N],Division[W],PutOuts,Assists,Errors,NewLeague[N]
1,1.0,315,81,7,24,38,39,14,3449,835,69,321,414,375,1.0,1.0,632,43,10,1.0
2,1.0,479,130,18,66,72,76,3,1624,457,63,224,266,263,0.0,1.0,880,82,14,0.0
3,1.0,496,141,20,65,78,37,11,5628,1575,225,828,838,354,1.0,0.0,200,11,3,1.0
4,1.0,321,87,10,39,42,30,2,396,101,12,48,46,33,1.0,0.0,805,40,4,1.0
5,1.0,594,169,4,74,51,35,11,4408,1133,19,501,336,194,0.0,1.0,282,421,25,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,1.0,497,127,7,65,48,37,5,2703,806,32,379,311,138,1.0,0.0,325,9,3,1.0
318,1.0,492,136,5,76,50,94,12,5511,1511,39,897,451,875,0.0,0.0,313,381,20,0.0
319,1.0,475,126,3,61,43,52,6,1700,433,7,217,93,146,0.0,1.0,37,113,7,0.0
320,1.0,573,144,9,85,60,78,8,3198,857,97,470,420,332,0.0,0.0,1314,131,12,0.0


In [10]:

sigma2 = OLS(Y,X).fit().summary()
print(sigma2)

#sigma^2 is variance estimated using the full model containing all predictors
sigma2 = OLS(Y,X).fit().scale 

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                     15.39
Date:                Sat, 24 Feb 2024   Prob (F-statistic):           7.84e-32
Time:                        16:50:10   Log-Likelihood:                -1876.2
No. Observations:                 263   AIC:                             3792.
Df Residuals:                     243   BIC:                             3864.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
intercept      163.1036     90.779      1.797   

In [11]:
residuals = OLS(Y,X).fit().resid
rss = np.sum(residuals**2)
rss

24200699.551662773

In [12]:
sigma2

99591.35617968219

In [13]:
neg_Cp = partial(nCp, sigma2)
neg_Cp

functools.partial(<function nCp at 0x000001AC2D36DE40>, 99591.35617968219)

In [14]:
strategy = Stepwise.first_peak(design,
                                direction='forward',
                                max_terms=10)
strategy

Strategy(initial_state=(), candidate_states=<bound method Stepwise.candidate_states of <ISLP.models.strategy.Stepwise object at 0x000001AC2D3CE890>>, build_submodel=<bound method ModelSpec.build_submodel of ModelSpec(terms=Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'NewLeague'],
      dtype='object'))>, check_finished=<function first_peak at 0x000001AC2B7893A0>, postprocess=<function _postprocess_best at 0x000001AC2B789620>)

In [20]:
hitters_MSE = sklearn_selected(OLS,
                                strategy)
hitters_MSE.fit(Hitters, Y)
hitters_MSE.selected_state_

('Assists',
 'AtBat',
 'CAtBat',
 'CRBI',
 'CRuns',
 'CWalks',
 'Division',
 'Hits',
 'PutOuts',
 'Walks')

In [16]:
 hitters_Cp = sklearn_selected(OLS,
                                strategy,
                                scoring=neg_Cp)
    
hitters_Cp.fit(Hitters, Y)
hitters_Cp.selected_state_


('Assists',
 'AtBat',
 'CAtBat',
 'CRBI',
 'CRuns',
 'CWalks',
 'Division',
 'Hits',
 'PutOuts',
 'Walks')